This notebook shows how to run a **Leela Chess Zero client on Google Colab's NVIDIA Tesla K80 GPU**. Thanks to [djinnome from GitHub who figured it out.](https://github.com/glinscott/leela-chess/issues/284).

Updated to work around problems with NVIDIA driver update: forces use of version 384.111.

Run each cell in order, waiting for the previous one to finish before running the next.

The scripts and cell layout may be messy/redundant, but this should work.


In [0]:
!wget http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1704/x86_64/cuda-repo-ubuntu1704_9.0.176-1_amd64.deb
!apt-get install -y --fix-missing --no-install-recommends dirmngr
!dpkg -i cuda-repo-ubuntu1704_9.0.176-1_amd64.deb
!apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1704/x86_64/7fa2af80.pub
!apt-get update
!mkdir /usr/lib/nvidia
!apt-get install -y --fix-missing --no-install-recommends linux-headers-generic nvidia-384=384.111-0ubuntu1 nvidia-opencl-dev nvidia-opencl-icd-384=384.111-0ubuntu1 opencl-headers
!apt-get install -y --fix-missing --no-install-recommends clinfo cmake git libboost-all-dev libopenblas-dev zlib1g-dev
!clinfo

The cells up to this point may take ~10 minutes to complete. You will know when it's done by the proper clinfo output showing the K80; also, the progress indicator will have stopped spinning.

In [0]:
!rm -rf leela-chess
!git clone https://github.com/glinscott/leela-chess.git
!cd leela-chess && git submodule update --init --recursive
!cd leela-chess && mkdir -p build

In [0]:
!cd leela-chess/build && cmake .. -DCMAKE_BUILD_TYPE=Release
!cd leela-chess/build && make -j $(nproc)

In [0]:
!cd leela-chess/build && curl -s -L https://github.com/glinscott/leela-chess/releases/latest | egrep -o '/glinscott/leela-chess/releases/download/v.*/client_linux' | head -n 1 | wget --base=https://github.com/ -i - -O client_linux && chmod +x client_linux

In [0]:
!echo '0;XgemmBatched;192;16;192;16; -DKWG=32 -DKWI=8 -DMDIMA=8 -DMDIMC=8 -DMWG=32 -DNDIMB=8 -DNDIMC=8 -DNWG=16 -DSA=1 -DSB=1 -DSTRM=0 -DSTRN=0 -DVWM=4 -DVWN=2;OpenCL: NVIDIA Corporation Tesla K80 @ 823MHz' > leela-chess/build/leelaz_opencl_tuning

In [0]:
# change user/password below if you want games credited to your account
# (user/password in original notebook were both 'XXX')
!cd leela-chess/build && ./client_linux --user 'Google Colab' --password googlecolab

If you get a **"signal: aborted (core dumped)" error** when running the client or **"failed to assign a backend" popup**, there are no GPUs available on Google Colab. **Try Runtime -> Restart Runtime** and running again, or kill the entire VM with **`!kill -9 -1`** and try again (VM may take 5 minutes to restart after being killed). As Google Colab has a limited number of free GPUs, **you may just have to try again another time**.



Changelog:
* 2018-05-11: no longer checkout from `next` - important training fix is now in `master`
* 2018-05-10: build lczero from `next` branch to pick up training fix
* 2018-06-02: use 384.111 NVIDIA driver to fix compatibility issues and update for 15 block NN